In [1]:
# 绑定谷歌硬盘，导入数据
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# 更改当前路径
import os
import pandas as pd
path = "/content/drive/My Drive/Movie Sentimental Analysis"
os.chdir(path)

# Any results you write to the current directory are saved as output.
import torch
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import random
import pickle
from tqdm import tqdm
from collections import Counter
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import time
from sklearn.metrics import accuracy_score

# 判断是否使用了gpu
import tensorflow as tf
tf.test.gpu_device_name()

# 读取数据
train = pd.read_csv('train.tsv', sep='\t')

# 标签数据
train_labels = torch.tensor(train['Sentiment'].values)   # 训练集标签

In [3]:
def Corpus_Extr(df):
    print('Construct Corpus...')
    corpus = []
    # 将phrase分为一个个词语
    for i in tqdm(range(len(df))):
        corpus.extend(df.Phrase[i].lower().split())
    corpus = Counter(corpus)
    corpus2 = sorted(corpus,key=corpus.get,reverse=True)
    print('Convert Corpus to Integers')
    # 构建word:int字典
    vocab_to_int = {word: idx for idx,word in enumerate(corpus2,1)}
    # 构建int:word字典
    int_to_vocab = {idx : word for idx,word in enumerate(corpus2,1)}
    print('Convert Phrase to Integers')
    phrase_to_int = []
    # 将phrase的词语列表替换为int列表
    for i in tqdm(range(len(df))):
        phrase_to_int.append([vocab_to_int[word] for word in df.Phrase.values[i].lower().split()])
    return corpus, vocab_to_int, int_to_vocab, phrase_to_int
corpus, vocab_to_int,int_to_vocab, phrase_to_int = Corpus_Extr(train)


# 对于不存在的词语用'<unk>'来表示
vocab_to_int['<unk>'] = 0
int_to_vocab[0] = '<unk>'

  3%|▎         | 5133/156060 [00:00<00:02, 51326.87it/s]

Construct Corpus...


  9%|▊         | 13507/156060 [00:00<00:01, 135068.84it/s]

Convert Corpus to Integers
Convert Phrase to Integers


100%|██████████| 156060/156060 [00:01<00:00, 115329.16it/s]


In [4]:
# 读取预训练的语言模型  glove-100
def read_glove():
    print("reading glove...")
    glove = {}
    with open("glove.txt", "r") as f:
        for line in f.readlines():
            line=line.split()
            word = line[0]
            vector = np.array(line[1:]).astype(np.float)
            # 如果该词语在词典中存在
            if vocab_to_int.get(word, -1) != -1:
                glove[vocab_to_int[word]] = vector
    # 定义glove[0] <unk>为0
    glove[0] = [0]*100
    return glove
# 读取glove模型
glove = read_glove()

# 将phrase padding 为相同的长度，多删少补
def Pad_sequences(phrase_to_int,seq_length):
    print("padding phrase")
    pad_sequences = np.zeros((len(phrase_to_int), seq_length),dtype=int)
    for idx,row in tqdm(enumerate(phrase_to_int),total=len(phrase_to_int)):
        pad_sequences[idx, :len(row)] = np.array(row)[:seq_length]
    return pad_sequences


reading glove...


In [51]:
# 构建model
class LSTM(nn.Module):
    def __init__(self,hidden_dim, linear_dim, n_word, n_layers,embed_dim, weight, labels):
        super().__init__()
        self.n_layers = n_layers    # lstm层数
        self.hidden_dim = hidden_dim    # 隐藏层节点数
        self.embedd_dim = embed_dim   # 使用glove的数据
        self.lables = labels # softmax要划分的类别数
        self.linear_dim = linear_dim # 线形层的节点数
        self.n_word = n_word # 词语总数
        
        # 定义词嵌入层，使用预训练glove模型来进行
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.embedding.weight.requires_grad = False

        # # 不使用glove，直接进行词语嵌入
        self.embedding = nn.Embedding(self.n_word, 100)

        # 定义双层的LSTM层
        self.lstm = nn.LSTM(self.embedd_dim, self.hidden_dim,num_layers = self.n_layers, bidirectional=True)

        # 定义线形层 使用relu激活函数 因为对lstm的输出进行了拼接，因此这里输入维度是hidden_dim*2
        self.linear = nn.Sequential(
            nn.Linear(self.hidden_dim * 2, self.linear_dim),
            nn.ReLU(True),
        )

        # 定义全连接层
        self.fc = nn.Linear(self.linear_dim, self.lables)

        
    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        # permute [batch_size, seq_len, embedded_size] -> [seq_len, batch_size, embedded_size]
        lstm_out, hidden = self.lstm(embeddings.permute([1, 0, 2]))
        output = self.linear(lstm_out[-1])
        output = self.fc(output)
        return output


In [70]:
labels = 5  # 要划分类别数
num_epochs = 5
embed_dim = 100
hidden_dim = 128
linear_dim = 64
n_layers = 2
batch_size = 64
lr = 0.001
device = torch.device('cuda:0')


# 读取glove模型
weight = torch.zeros(len(vocab_to_int), embed_dim)
count = 0 
for i in range(len(vocab_to_int)):
        # 若该词语不存在，通<unk>的值
        if i not in glove.keys():
            count+=1
        weight[i, :] = torch.Tensor(glove.get(i,np.random.rand(1,100)))
    
# 整理训练数据
train_features = torch.LongTensor(Pad_sequences(phrase_to_int,30))

train_set = torch.utils.data.TensorDataset(train_features, train_labels)
#train_iter = torch.utils.data.DataLoader(train_set, batch_size=batch_size,shuffle=True)
train_iter = torch.utils.data.DataLoader(train_set, batch_size=batch_size)

net = LSTM(hidden_dim, linear_dim, len(vocab_to_int),n_layers,embed_dim, weight, labels)

net.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr)
# 动态调整学习率，每隔step_size就将学习率调整为0.1倍
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 40, gamma=0.1, last_epoch=-1)


 27%|██▋       | 42127/156060 [00:00<00:00, 421250.31it/s]

padding phrase



100%|██████████| 156060/156060 [00:00<00:00, 413783.37it/s]


In [ ]:
# 训练模型
from torch.autograd import Variable
global_batch_size = 0 # 全局的batchnum统计
for epoch in range(num_epochs):
    start = time.time()
    train_loss, test_losses = 0, 0
    train_acc, test_acc = 0, 0
    n, m = 0, 0
    batch_num = 0
    for feature, label in train_iter:
        n += 1
        batch_num +=1
        global_batch_size+=1
        net.zero_grad()
        feature = Variable(feature.cuda())
        label = Variable(label.cuda())
        score = net(feature)
        loss = loss_function(score, label)
        loss.backward()
        optimizer.step()
        train_acc += accuracy_score(torch.argmax(score.data.cpu(), dim=1), label.cpu())
        train_loss += loss
        
        if batch_num %100 ==0:
            print('batch_num: %d, train loss: %.4f, train acc: %.2f'%
                        (batch_num, train_loss / n, train_acc / n))
            
    scheduler.step()    # 更新学习率
    cur_lr = optimizer.param_groups[0]['lr']    # 查看当前学习率
    end = time.time()
    runtime = end - start
    print('epoch: %d, train loss: %.4f, train acc: %.2f, time: %.2f, learning rate: %f' %
          (epoch, train_loss / n, train_acc / n, runtime, cur_lr))

torch.save(net, 'network.pkl')

In [ ]:
plt.figure()
plt.plot(x_batch,y64_loss, label='batch_size=64')
plt.plot(x_batch,y128_loss, label='batch_size=128')
plt.plot(x_batch,y256_loss, label='batch_size=256')
plt.legend()
plt.xlabel('batch_num')
plt.ylabel('loss')
plt.savefig('training_batch_size.png', format='png')

In [ ]:
# 绘制图像
import matplotlib.pyplot as plt 
# 损失函数
plt.figure()
plt.plot(x_epoch,y_loss, color='m')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.savefig('training_loss.png', format='png')

# 精确度
plt.figure()
plt.plot(x_epoch,y_acc, color='y')
plt.xlabel('epoch')
plt.ylabel('acc')
plt.savefig('training_acc.png', format='png')

# 学习率变化情况
plt.figure()
plt.plot(x_epoch,y_lr)
plt.xlabel('epoch')
plt.ylabel('lr')
plt.savefig('training_lr.png', format='png')

In [ ]:
import csv
# 测试数据集
f = open('submission.csv', 'w', newline='') # 要写入的测试文件
writer = csv.writer(f)
writer.writerow(['PhraseId', 'Sentiment'])
net = torch.load('network.pkl').cuda()  # 加载模型

# 读取测试数据
df = pd.read_csv('test.tsv', sep='\t')
test_phrase_to_int = []
# 将phrase的词语列表替换为int列表   如果词语不存在则用0代替
for i in range(len(df)):
    test_phrase_to_int.append([vocab_to_int.get(word, 0) for word in df.Phrase.values[i].lower().split()])
# 对数据进行padding
test_phrase_to_int = Pad_sequences(test_phrase_to_int,30)

# 遍历测数据计算结果
id = 156061
for i in range(len(test_phrase_to_int)):
    if i % 3000 ==0:
        print("======>",i)
    feature = torch.LongTensor(test_phrase_to_int[i])
    feature = torch.unsqueeze(feature, dim=0)
    feature = Variable(feature.cuda())
    score = net(feature)
    res = torch.argmax(score.data.cpu())
    writer.writerow([id, res.item()])
    id += 1
f.close()
    